In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import colors

from read_data import ReadClass

KE_spec_series = np.load('./data/KE_spec_series_full.npy')
PE_spec_series = np.load('./data/PE_spec_series_full.npy')

KE_spec_1 = np.load('./data/KE_spec_series_1.npy')
KE_spec_2 = np.load('./data/KE_spec_series_2.npy')
KE_spec_3 = np.load('./data/KE_spec_series_3.npy')

PE_spec_1 = np.load('./data/PE_spec_series_1.npy')
PE_spec_2 = np.load('./data/PE_spec_series_2.npy')
PE_spec_3 = np.load('./data/PE_spec_series_3.npy')

freq_axis_1 = np.load('./data/freq_axis_1.npy')
freq_axis_2 = np.load('./data/freq_axis_2.npy')
freq_axis_3 = np.load('./data/freq_axis_3.npy')

delta_freq_1 = freq_axis_1[1] - freq_axis_1[0]
delta_freq_2 = freq_axis_2[1] - freq_axis_2[0]
delta_freq_3 = freq_axis_3[1] - freq_axis_3[0]
print(delta_freq_1, delta_freq_2, delta_freq_3)

experiment_name = "exRo9N10e01_LES"
rc('text', usetex=True)
rc('font', family='serif')
fs = 14
thin_data = 32
thin_time = 32

figure_directory = "./figure"

reader = ReadClass(experiment_name)
reader.read_geometry(0)
time_days = reader.time / 86400
NT = np.size(time_days)

freq_vortex = np.sqrt(reader.alpha_plus * reader.alpha_minus)
freq_vortex_days = freq_vortex * 86400
freq_buoyancy = reader.buoyancy_frequency * 86400
freq_inertia = (
    reader.angular_velocity[2]*2 - reader.alpha_plus - reader.alpha_minus
) * 86400
period_vortex = 2 * np.pi / freq_vortex
MT = round(period_vortex / reader.output_interval)

In [ ]:
color = [(1.0, 1.0, 1.0), (0.0, 0.6, 1.0), (0.0, 0.4, 0.6), (0.0, 0.7, 0.5),
         (1.0, 1.0, 0.0), (1.0, 0.5, 0.0), (1.0, 0.0, 0.0), (0.0, 0.0, 0.0)]
cmap_name = 'cbgyrk'
cm_cbgyrk = colors.LinearSegmentedColormap.from_list(cmap_name, color, N=300)

nt_start = MT * 20
nt_end = MT * 150
nt_interval = nt_start
nTime = nt_end - nt_start

time_axis = np.arange(nt_start, nt_end) - nt_interval // 2
time_axis = time_axis * reader.output_interval / 86400

delta_freq = 2 * np.pi / (reader.output_interval * nt_interval)
freq_axis = np.arange(0, nt_interval // 2 + 1)
freq_axis = freq_axis * delta_freq * 86400

levels = np.logspace(-8.5, -3.5, 50, endpoint=True)
ticks = np.logspace(-8, -4, 5)

fig = plt.figure(figsize=[10, 6])
fig.subplots_adjust(left=0.06, bottom=0.08, right=0.99,
                    top=0.98, wspace=0.38, hspace=0.11)

ax1 = fig.add_subplot(221)
c1 = ax1.contourf(
    time_axis[::thin_time], freq_axis,
    KE_spec_series[:, ::thin_time] / (delta_freq * 86400),
    cmap=cm_cbgyrk, levels=levels,
    norm=colors.LogNorm(), extend="both"
)
ax1.set_yscale('log')
ax1.set_ylabel(r'$\omega$ [rad/day]', fontsize=14)
ax1.set_ylim(freq_axis[1]*0.8, freq_axis[-1])

cax = fig.add_axes((0.465, 0.3, 0.01, 0.4))
cbar = fig.colorbar(c1, cax=cax, extend='both', ticks=ticks)
cbar.ax.set_yticklabels(
    [r'$10^{-8}$', r'$10^{-7}$', r'$10^{-6}$', r'$10^{-5}$', r'$10^{-4}$'])
cbar.set_label(r'[J$\cdot$day/kg]', labelpad=18,
               y=0.5, rotation=-90, fontsize=fs)

ax2 = fig.add_subplot(222)
ax2.plot(freq_axis_1, KE_spec_1[:] / delta_freq_1,
         'g', label='{}-{}days'.format(40, 80))
ax2.plot(freq_axis_2, KE_spec_2[:] / delta_freq_2,
         'c', label='{}-{}days'.format(80, 125))
ax2.plot(freq_axis_3, KE_spec_3[:] / delta_freq_3,
         'm', label='{}-{}days'.format(160, 220))
nTime = 100 * 50
delta_freq = 2 * np.pi / (reader.output_interval * nTime) * 86400
freq_axis_GM = np.logspace(-3, 3, 1000, endpoint=True) + freq_inertia
ax2.plot(freq_axis_GM,
         5.e-6 / (freq_axis_GM**2 - freq_inertia**2)**0.5 / freq_axis_GM,
         'k')
ax2.vlines(freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax2.vlines(2 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax2.vlines(3 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax2.vlines(0.4 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax2.vlines(0.6 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)

ax2.vlines(freq_buoyancy, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax2.vlines(freq_inertia, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax2.legend()

ax2.set_xlim(2.e-1, 1.8e2)
ax2.set_ylim(1.e-12, 1.e-3)
ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.set_ylabel(r'[J$\cdot$day/kg]', fontsize=14);
grid_y_ticks_major = 10.0**np.arange(-12, -2, 1)
ax2.set_yticks(grid_y_ticks_major, major=True)

delta_freq = 2 * np.pi / (reader.output_interval * nt_interval)

ax3 = fig.add_subplot(223)
c2 = ax3.contourf(
    time_axis[::thin_time], freq_axis,
    PE_spec_series[:, ::thin_time] / (delta_freq * 86400),
    cmap=cm_cbgyrk, levels=levels,
    norm=colors.LogNorm(), extend="both"
)

ax3.set_yscale('log')
ax3.set_xlabel(r'$t$ [days]', fontsize=14)
ax3.set_ylabel(r'$\omega$ [rad/day]', fontsize=14)
ax3.set_ylim(freq_axis[1]*0.8, freq_axis[-1])

ax4 = fig.add_subplot(224)
ax4.plot(freq_axis_1, PE_spec_1[:] / delta_freq_1,
         'g', label='{}-{}days'.format(40, 80))
ax4.plot(freq_axis_2, PE_spec_2[:] / delta_freq_2,
         'c', label='{}-{}days'.format(80, 125))
ax4.plot(freq_axis_3, PE_spec_3[:] / delta_freq_3,
         'm', label='{}-{}days'.format(160, 220))

ax4.vlines(freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax4.vlines(2 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax4.vlines(3 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax4.vlines(0.4 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax4.vlines(0.6 * freq_vortex_days, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)

ax4.vlines(freq_buoyancy, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax4.vlines(freq_inertia, ymin=0, ymax=10,
           colors='grey', linestyles='--', linewidths=1)
ax4.legend()

ax4.set_yscale('log')
ax4.set_xscale('log')
ax4.set_xlim(2.e-1, 1.8e2)
ax4.set_ylim(1.e-12, 1.e-3)
grid_y_ticks_major = 10.0**np.arange(-12, -2, 1)
ax4.set_yticks(grid_y_ticks_major, major=True)

ax4.set_ylabel(r'[J$\cdot$day/kg]', fontsize=14);
ax4.set_xlabel(r'$\omega$ [rad/day]', fontsize=14);

figure_name = "frequency_series"
fig.savefig(figure_directory + "/" + figure_name + '.eps')
fig.savefig(figure_directory + "/" + figure_name + '.png', dpi=300)
